# OPTAA Calibration Script
This script provides an example of using the OPTAA Calibration parser for writing a calibration csv in the appropriate manner for OOInet to be pushed to asset management. 

In [1]:
from utils import *

In [2]:
import shutil
import string

In [24]:
def get_calibration_files(serial_nums,dirpath):
    """
    Function which gets all the calibration files associated with the
    instrument serial numbers.
    
    Args:
        serial_nums - serial numbers of the instruments
        dirpath - path to the directory containing the calibration files
    Returns:
        calibration_files - a dictionary of instrument uids with associated
            calibration files
    """
    calibration_files = {}
    for uid in serial_nums.keys():
        sn = serial_nums.get(uid)[0]
        sn = str(sn)
        files = []
        for file in os.listdir(dirpath):
            if sn in file:
                if 'Calibration_File' in file:
                    files.append(file)
                else:
                    pass
            else:
                pass
        
        calibration_files.update({uid:files})
        
    return calibration_files

In [4]:
def ensure_dir(file_path):
    """
    Function which checks that the directory where you want
    to save a file exists. If it doesn't, it creates the 
    directory.
    """
    if not os.path.exists(file_path):
        os.makedirs(file_path)

In [5]:
class OPTAACalibration():
    
    def __init__(self, uid):
        self.serial = ''
        self.uid = uid
        self.date = None
        self.cwlngth = []
        self.awlngth = []
        self.tcal = None
        self.tbins = None
        self.ccwo = []
        self.acwo = []
        self.tcarray = []
        self.taarray = []
        self.nbins = None  # number of temperature bins
        self.coefficients = {'CC_taarray': 'SheetRef:CC_taarray',
                             'CC_tcarray': 'SheetRef:CC_tcarray'}
        
    @property
    def uid(self):
        return self._uid
        
    @uid.setter
    def uid(self, d):
        r = re.compile('.{5}-.{6}-.{5}')
        if r.match(d) is not None:
            self.serial = 'ACS-' + d.split('-')[2].strip('0')
            self._uid = d       
        else:
            raise Exception(f"The instrument uid {d} is not a valid uid. Please check.")
            

    def load_dev(self, filepath):
        """
        Function loads the dev file for the OPTAA.
        
        Args:
            filepath - the full path, including the name of the file, to the optaa
                dev file.
        Returns:
            self.date - the date of calibration
            self.tcal - calibration temperature
            self.nbins - number of temperature bins
            self.cwlngth
            self.awlngth
            self.ccwo
            self.acwo
            self.tcarray
            self.taarray
            self.coefficients - a dictionary of the calibration values and associated
                keys following the OOI csv naming convention

        """
        
        
        if filepath.endswith('.zip'):
            with ZipFile(filepath) as zfile:
                filename = [name for name in zfile.namelist() if name.endswith('.dev')]
                text = zfile.read(filename[0]).decode('ASCII')
        
        else:
            with open(filepath) as file:
                text = file.read()
            
        # Remove extraneous characters from the 
        punctuation = ''.join((letter for letter in string.punctuation if letter not in ';/.'))
        
        for line in text.replace('\t',' ').splitlines():
            line = ''.join((word for word in line if word not in punctuation))
            
            if 'tcal' in line:
                data = line.split()
                # Temperature calibration value
                tcal = data.index('tcal')
                self.tcal = data[tcal+1]
                self.coefficients['CC_tcal'] = self.tcal
                # Temperature calibration date
                cal_date = data[-1].strip()
                self.date = pd.to_datetime(cal_date).strftime('%Y%m%d')
                
            elif ';' in line:
                data, comment = line.split(';')
        
                if 'temperature bins' in comment:
                    if 'number' in comment:
                        self.nbins = int(data)
                    else:
                        self.tbins = data.split()
                        self.tbins = [float(x) for x in self.tbins]
                        self.coefficients['CC_tbins'] = json.dumps(self.tbins)
                
                elif 'C and A offset' in comment:
                    data = data.split()
                    self.cwlngth.append(float(data[0][1:]))
                    self.awlngth.append(float(data[1][1:]))
                    self.ccwo.append(float(data[3]))
                    self.acwo.append(float(data[4]))
                    tcrow = [float(x) for x in data[5:self.nbins+5]]
                    tarow = [float(x) for x in data[self.nbins+5:2*self.nbins+5]]
                    self.tcarray.append(tcrow)
                    self.taarray.append(tarow)
                    self.coefficients['CC_cwlngth'] = json.dumps(self.cwlngth)
                    self.coefficients['CC_awlngth'] = json.dumps(self.awlngth)
                    self.coefficients['CC_ccwo'] = json.dumps(self.ccwo)
                    self.coefficients['CC_acwo'] = json.dumps(self.acwo)
            
                else:
                    pass
            
            else:
                pass
            
            
    def write_csv(self, savepath):
        """
        This function writes the correctly named csv file for the ctd to the
        specified directory.
    
        Args:
            outpath - directory path of where to write the csv file
        Raises:
            ValueError - raised if the OPTAA's object's coefficient dictionary 
                has not been populated
        Returns:
            self.to_csv - a csv of the calibration coefficients which is 
                written to the specified directory from the outpath.
        """
        # Now, write to a csv file
        # Create a dataframe to write to the csv
        data = {
            'serial':self.serial,
            'name':list(self.coefficients.keys()),
            'value':list(self.coefficients.values()),
            'notes':['']*len(self.coefficients)
        }
        
        df = pd.DataFrame().from_dict(data)
        
        # Generate the cal csv filename
        filename = self.uid + '__' + self.date + '.csv'
        # Now write to 
        check = input(f"Write {filename} to {savepath}? [y/n]: ")
        if check.lower().strip() == 'y':
            df.to_csv(savepath+'/'+filename, index=False)
        
        # Generate the tc and ta array filename
        tc_name = filename + '__CC_tcarray.ext'
        ta_name = filename + '__CC_taarray.ext'
        
        def write_array(filename, array):
            with open(filename, 'w') as out:
                array_writer = csv.writer(out)
                array_writer.writerows(array)
                
        write_array(savepath+'/'+tc_name, self.tcarray)
        write_array(savepath+'/'+ta_name, self.taarray)

These are my **local** directories where the calibration, qct, and whoi asset management records are saved. These will be different on your machine. I highly suggests putting in the full, absolute path of at least the base directories where your information is stored.

In [22]:
qct_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/'
cal_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/OPTAA/'
asset_management_directory = '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/OPTAA/'

This is the full directory path to the local copy of the WHOI OOI-CGSN asset tracking spreadsheet. I call the function whoi_asset_tracking which will load the instrument class we are interested in, which in this case is the OPTAAs.

In [9]:
excel_spreadsheet = '/media/andrew/OS/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
sheet_name = 'Sensors'

Below, we load the whoi asset tracking spreadsheet. This gives us a guide on how to search for specific calibration files, qct files, deployment information, etc for specific instruments.

In [16]:
OPTAA = whoi_asset_tracking(excel_spreadsheet,sheet_name,instrument_class='OPTAA')
OPTAA.head(5)

,Instrument Class,Series,Supplier Serial Number,WHOI #,OOI #,UID,Model,CGSN PN,Firmware Version,Supplier,...,QCT Testing,PreDeployment,Post Deployment,Refurbishment/ Repair,DO Number,Date Received,Deployment History,Current Deployment,Instrument Location on Current Deployment,Notes
932,OPTAA,D,123,,A00098,CGINS-OPTAAD-00123,AC-S,1336-00007-00004,1.11,WET Labs,...,3305-00113-00002\n3305-00113-00011\n3305-00113...,3305-00313-00034\n3305-00313-00102\n3305-00313...,3305-00513-00013\n3305-00513-00061,3305-00900-00009\n3305-00900-00129\n3305-00900...,WH-SC12-17-OPTAA-1001,12/3/2012\n07/20/15,CP01CNSM-00001\nCP03ISSM-00003\nCP01CNSM-00007...,NaN,NaN,NaN
933,OPTAA,D,129,115289,A00178,CGINS-OPTAAD-00129,AC-S,1336-00007-00004,1.1,WET Labs,...,3305-00113-00005\n3305-00113-00067\n3305-00113...,3305-00313-00056\n3305-00313-00159,3305-00513-00024,3305-00900-00009\n3305-00900-00170\n3305-00900...,WH-SC12-17-OPTAA-1001,1/29/2013\n10/05/15,CP01CNSM-00001\nCP01CNSM-00005,CP03ISSM-00009,NSIF,09-27-2013 Pump 05-6836 returned to Wetlab for...
934,OPTAA,D,130,115290,A00179,CGINS-OPTAAD-00130,AC-S,1336-00007-00004,1.1,WET Labs,...,3305-00113-00006\n3305-00113-00044\n3305-00113...,3305-00313-00021\n3305-00313-00089,NaN,3305-00900-00001\n3305-00900-00069,WH-SC12-17-OPTAA-1001,1/29/2013\n12/12/2014,CP03ISSM-00002\nGS01SUMO-00003,NaN,NaN,09-27-2013 Pump 05-6838 returned to Wetlab for...
935,OPTAA,D,150,115838,A00496,CGINS-OPTAAD-00150,AC-S,1336-00007-00004,1.11,WET Labs,...,3305-00113-00014\n3305-00113-00088,NaN,NaN,3305-00900-00068,WH-SC12-17-OPTAA-1003,2013-12-12 00:00:00,GI01SUMO-00001\nGS01SUMO-00003,NaN,NaN,(NSIF)
936,OPTAA,D,151,115839,A00497,CGINS-OPTAAD-00151,AC-S,1336-00007-00004,1.11,WET Labs,...,3305-00113-00015\n3305-00113-00083\n3305-00113...,3305-00313-00007\n3305-00313-00029\n3305-00313...,3305-00513-00027\n3305-00513-00065,3305-00900-00069\n3305-00900-00170\n3305-00900...,WH-SC12-17-OPTAA-1003,2013-12-12 00:00:00,CP01CNSM-00002\nCP01CNSM-00003\nCP03ISSM-00004...,NaN,NaN,NaN


We can get a list of the unique OPTAA UIDs, which we will need to get the serial numbers

In [12]:
uids = list(set(OPTAA['UID']))
# print(uids)

With the whoi_asset_tracking info for the OPTAAs as well as their UIDs, we can call the get_serial_nums function to return the serial numbers for each OPTAA uid.

In [13]:
serial_nums = get_serial_nums(OPTAA, uids)
# print(serial_nums)

The function below will return a dictionary of all the QCT Testing document numbers for each UID:

In [20]:
qct_dict = {}
for uid in uids:
    # Get the QCT Document numbers from the asset tracking sheet
    OPTAA['UID_match'] = OPTAA['UID'].apply(lambda x: True if uid in x else False)
    qct_series = OPTAA[OPTAA['UID_match'] == True]['QCT Testing']
    qct_series = list(qct_series.iloc[0].split('\n'))
    qct_dict.update({uid:qct_series})

In [25]:
cal_dict = get_calibration_files(serial_nums, cal_directory)
cal_dict

{'CGINS-OPTAAD-00130': ['OPTAA-D_AC-S_SN_130_Calibration_Files_2013-01-30.zip',
  'OPTAA-D_AC-S_SN_130_Calibration_Files_2014-12-30.zip',
  'OPTAA-D_AC-S_SN_130_Calibration_Files_2016-06-01.zip'],
 'CGINS-OPTAAD-00187': ['OPTAA-D_AC-S_SN_187_Calibration_Files_2014-09-26.zip',
  'OPTAA-D_AC-S_SN_187_Calibration_Files_2016-06-01.zip',
  'OPTAA-D_AC-S_SN_187_Calibration_Files_2018-12-18.zip'],
 'CGINS-OPTAAD-00153': ['OPTAA-D_AC-S_SN_153_Calibration_Files_2013-12-12.zip',
  'OPTAA-D_AC-S_SN_153_Calibration_Files_2016-08-16.zip'],
 'CGINS-OPTAAD-00241': ['OPTAA-D_AC-S_SN_241_Calibration_Files_2015-04-07.zip',
  'OPTAA-D_AC-S_SN_241_Calibration_Files_2016-03-31.zip',
  'OPTAA-D_AC-S_SN_241_Calibration_Files_2017-09-06.zip'],
 'CGINS-OPTAAD-00255': ['OPTAA-D_AC-S_SN_255_Calibration_Files_2015-08-04.zip',
  'OPTAA-D_AC-S_SN_255_Calibration_Files_2017-07-11.zip',
  'OPTAA-D_AC-S_SN_255_Calibration_Files_2018-11-15.zip'],
 'CGINS-OPTAAD-00150': ['OPTAA-D_AC-S_SN_150_Calibration_Files_2013-12-12

In [26]:
# Lets try opening/loading
uid = uids[0]
uid

'CGINS-OPTAAD-00130'

In [27]:
cal_dict[uid]

['OPTAA-D_AC-S_SN_130_Calibration_Files_2013-01-30.zip',
 'OPTAA-D_AC-S_SN_130_Calibration_Files_2014-12-30.zip',
 'OPTAA-D_AC-S_SN_130_Calibration_Files_2016-06-01.zip']

In [33]:
filepath = generate_file_path(cal_directory, cal_dict[uid][-1], ext=['.zip'])

In [32]:
def generate_file_path(dirpath,filename,ext=['.cap','.txt','.log'],exclude=['_V','_Data_Workshop']):
    """
    Function which searches for the location of the given file and returns
    the full path to the file.
    
    Args:
        dirpath - parent directory path under which to search
        filename - the name of the file to search for
        ext - file endings to search for
        exclude - optional list which allows for excluding certain
            directories from the search
    Returns:
        fpath - the file path to the filename from the current
            working directory.
    """
    # Check if the input file name has an extension already
    # If it does, parse it for input into the search algo
    if '.' in filename:
        check = filename.split('.')
        filename = check[0]
        ext = ['.'+check[1]]

    for root, dirs, files in os.walk(dirpath):
        dirs[:] = [d for d in dirs if d not in exclude]
        for fname in files:
            if fnmatch.fnmatch(fname, [filename+'*'+x for x in ext]):
                fpath = os.path.join(root, fname)
                return fpath

In [34]:
filepath

'/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/OPTAA/OPTAA-D_AC-S_SN_130_Calibration_Files_2016-06-01.zip'

In [ ]:
import string

In [ ]:
optaa = OPTAACalibration(uid=uid)

In [ ]:
optaa.load_dev(filepath)

In [ ]:
optaa.write_csv(optaa_savepath)

In [ ]:
import shutil

In [ ]:
# Purge the temp directory
try:
    shutil.rmtree('/'.join((os.getcwd(),'temp')))
except:
    pass
# Put the csv files into a similar temp directory for local working
ensure_dir('/'.join((os.getcwd(),'temp','optaa')))
optaa_savepath = '/'.join((os.getcwd(),'temp','optaa'))

In [ ]:
optaa_savepath

In [ ]:
optaa.coefficients

In [ ]:
OPTAA = whoi_asset_tracking(spreadsheet=excel_spreadsheet,sheet_name=sheet_name,instrument_class='OPTAA')